In [19]:
import numpy as np
import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

consumerTrain = pd.read_csv("train_dataset.csv")
consumerTest = pd.read_csv("test_dataset.csv")

Id = consumerTest["用户编码"]
# consumerTest = consumerTest.drop(["用户编码"],axis=1)


print("-"*50+ "train简单查看统计学信息" +"-"*50)
print(consumerTrain.info())
print("-"*50+ "test简单查看统计学信息" +"-"*50)
print(consumerTest.info())

print("-"*50 + "train查看缺失值" + "-"*50)
print(consumerTrain.isnull().sum().sort_values(ascending=False).head(10))
print("-"*50 + "test查看缺失值" + "-"*50)
print(consumerTest.isnull().sum().sort_values(ascending=False).head(10))


--------------------------------------------------train简单查看统计学信息--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 30 columns):
用户编码               50000 non-null object
用户实名制是否通过核实        50000 non-null int64
用户年龄               50000 non-null int64
是否大学生客户            50000 non-null int64
是否黑名单客户            50000 non-null int64
是否4G不健康客户          50000 non-null int64
用户网龄（月）            50000 non-null int64
用户最近一次缴费距今时长（月）    50000 non-null int64
缴费用户最近一次缴费金额（元）    50000 non-null float64
用户近6个月平均消费值（元）     50000 non-null float64
用户账单当月总费用（元）       50000 non-null float64
用户当月账户余额（元）        50000 non-null int64
缴费用户当前是否欠费缴费       50000 non-null int64
用户话费敏感度            50000 non-null int64
当月通话交往圈人数          50000 non-null int64
是否经常逛商场的人          50000 non-null int64
近三个月月均商场出现次数       50000 non-null int64
当月是否逛过福州仓山万达       50000 non-null int64
当月是否到过福州山姆会员店      50000 non-null int64
当月是否看电影  

In [20]:
consumerTrain["次数"] = consumerTrain["当月网购类应用使用次数"]+consumerTrain["当月物流快递类应用使用次数"]+consumerTrain["当月金融理财类应用使用总次数"]+consumerTrain["当月视频播放类应用使用次数"]+consumerTrain["当月飞机类应用使用次数"]+consumerTrain["当月火车类应用使用次数"]+consumerTrain["当月旅游资讯类应用使用次数"]
consumerTest["次数"] = consumerTest["当月网购类应用使用次数"]+consumerTest["当月物流快递类应用使用次数"]+consumerTest["当月金融理财类应用使用总次数"]+consumerTest["当月视频播放类应用使用次数"]+consumerTest["当月飞机类应用使用次数"]+consumerTest["当月火车类应用使用次数"]+consumerTest["当月旅游资讯类应用使用次数"]
            
consumerTrain["火车飞机类应用使用次数"] = consumerTrain["当月飞机类应用使用次数"]+consumerTrain["当月火车类应用使用次数"]
consumerTest["火车飞机类应用使用次数"] = consumerTest["当月飞机类应用使用次数"]+consumerTest["当月火车类应用使用次数"]

consumerTrain["是否不良"] = consumerTrain["是否黑名单客户"]+consumerTrain["是否4G不健康客户"]
consumerTest["是否不良"] = consumerTest["是否黑名单客户"]+consumerTest["是否4G不健康客户"]

consumerTrain["是否不良"][consumerTrain["是否不良"]>1] = 1
consumerTest["是否不良"][consumerTest["是否不良"]>1] = 1

In [21]:
def simple_features(df_data):
    df = df_data
    df["次数"] = df["当月网购类应用使用次数"]+df["当月物流快递类应用使用次数"]+df["当月金融理财类应用使用总次数"]+df["当月视频播放类应用使用次数"]+df["当月飞机类应用使用次数"]+df["当月火车类应用使用次数"]+df["当月旅游资讯类应用使用次数"]

    df["火车飞机类应用使用次数"] = df["当月飞机类应用使用次数"]+df["当月火车类应用使用次数"]

    df["是否不良"] = df["是否黑名单客户"]+df["是否4G不健康客户"]

    df["是否不良"][df["是否不良"]>1] = 1
    
    for col in ['当月金融理财类应用使用总次数','当月旅游资讯类应用使用次数']: # 这两个比较积极向上一点
        df[col + '百分比'] = df[col].values / df['次数'].values 
    
    
    df['当月通话人均话费'] = df['用户账单当月总费用（元）'].values / (df['当月通话交往圈人数'].values + 1)
    df['上个月费用'] = df['用户当月账户余额（元）'].values + df['用户账单当月总费用（元）'].values
     
    df['用户上网年龄'] = df['用户年龄'] - df['用户网龄（月）']
    df['用户上网年龄百分比'] = df['用户网龄（月）'] / (df['用户年龄'] + 1)
     
    df['近似总消费'] = df['用户近6个月平均消费值（元）'] / 6 * df['用户网龄（月）']
    
    df = df.fillna(0)
    return df
    

In [22]:
consumerTrain = simple_features(consumerTrain)
consumerTest = simple_features(consumerTest)
fea_cols = [col for col in consumerTrain.columns if consumerTrain[col].dtypes!='object' and consumerTrain[col].dtypes != '<M8[ns]' and col!='用户编码' and col!='信用分']   

In [23]:
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import StratifiedKFold,KFold
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
import gc

def _get_values_lgbregresser_models(df_fea, df_label,  feature_names):
    kf = KFold(n_splits=5,shuffle=False)#,random_state=1)
     
    models  = []
    models_1 = []
    models_2 = []
    
    importances = pd.DataFrame() 
    
    lgb_params = {'num_leaves': 31,
         'min_data_in_leaf': 32, 
#          'objective':'mae',
         'max_depth': -1,
         'learning_rate': 0.005,2
         "min_child_samples": 20,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         'n_estimators': 10000,
         "bagging_seed": 11,
         "metric": 'rmse',
         "lambda_l1": 0.1,
         "nthread": 50,
         "verbosity": -1}


    lgb_params1 = {'num_leaves': 31,
         'min_data_in_leaf': 32, 
         'objective':'mae',
         'max_depth': -1,
         'learning_rate': 0.005,
         "min_child_samples": 20,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         'n_estimators': 10000,
         "bagging_seed": 11,
         "lambda_l1": 0.1,
         "nthread": 50,
         "verbosity": -1}
    
    min_val = np.min(df_label)
    print(min_val)
    for fold_, (trn_, val_) in enumerate(kf.split(df_fea)): 
        trn_x, trn_y= df_fea[trn_,:], df_label[trn_]#, df_label1[trn_] 
        val_x, val_y = df_fea[val_,:], df_label[val_]#, df_label1[val_] 
        tmp = pd.DataFrame()
         
        
        model = lgb.LGBMRegressor(**lgb_params1)
        model.fit(trn_x, trn_y, eval_set=[(trn_x, trn_y), (val_x, val_y)], eval_metric ='mae',
                  verbose=50,early_stopping_rounds=250)     
        tmp['target'] = val_y
        tmp['pred1'] = model.predict(val_x)
        models.append(model)
        
        model1 = lgb.LGBMRegressor(**lgb_params)
        model1.fit(trn_x, trn_y, eval_set=[(trn_x, trn_y), (val_x, val_y)], eval_metric ='mae',
                   verbose=50,early_stopping_rounds=250)     
        tmp['pred2'] = model1.predict(val_x)
        models_1.append(model1)
  
        tmp = tmp.sort_values('pred1')
        tmp['ranks'] = list(range(tmp.shape[0]))
        tmp['preds'] = tmp['pred1'].values
        tmp.loc[tmp.ranks<2000,'preds']  = tmp.loc[tmp.ranks< 2000,'pred2'].values *0.4 + tmp.loc[tmp.ranks< 2000,'pred1'].values * 0.6
        tmp.loc[tmp.ranks>8000,'preds']  = tmp.loc[tmp.ranks> 8000,'pred2'].values *0.4 + tmp.loc[tmp.ranks> 8000,'pred1'].values * 0.6
         
        print('*' * 100)
        print('MAE Model',     1 / (1 + (mean_absolute_error(y_true= tmp['target'] , y_pred= tmp['pred1'] ))))
        print('MSE Model',     1 / (1 + (mean_absolute_error(y_true= tmp['target'] , y_pred= tmp['pred2'] ))))
        print('Merge Model12', 1 / (1 + (mean_absolute_error(y_true= tmp['target'] , y_pred= tmp['preds'] )))) 
        
        imp_df = pd.DataFrame()
        imp_df['feature'] = feature_names
        imp_df['gain'] = model.feature_importances_
        imp_df['fold'] = fold_ + 1
        
        importances = pd.concat([importances, imp_df], axis=0)
        
        gc.collect() 
    return models,models_1,importances 

In [24]:

models_mae, models_mse, importances   = _get_values_lgbregresser_models(consumerTrain[fea_cols].values, 
                                                                        consumerTrain['信用分'].values, feature_names=fea_cols)

422
Training until validation scores don't improve for 250 rounds.
[50]	valid_0's l1: 26.783	valid_1's l1: 27.0031
[100]	valid_0's l1: 22.7205	valid_1's l1: 22.9447
[150]	valid_0's l1: 20.0321	valid_1's l1: 20.2956
[200]	valid_0's l1: 18.2707	valid_1's l1: 18.5683
[250]	valid_0's l1: 17.1241	valid_1's l1: 17.4793
[300]	valid_0's l1: 16.3701	valid_1's l1: 16.7888
[350]	valid_0's l1: 15.8714	valid_1's l1: 16.342
[400]	valid_0's l1: 15.5293	valid_1's l1: 16.0394
[450]	valid_0's l1: 15.2855	valid_1's l1: 15.8256
[500]	valid_0's l1: 15.0961	valid_1's l1: 15.6616
[550]	valid_0's l1: 14.9475	valid_1's l1: 15.5409
[600]	valid_0's l1: 14.8273	valid_1's l1: 15.4458
[650]	valid_0's l1: 14.7261	valid_1's l1: 15.3701
[700]	valid_0's l1: 14.6394	valid_1's l1: 15.3106
[750]	valid_0's l1: 14.5621	valid_1's l1: 15.261
[800]	valid_0's l1: 14.4977	valid_1's l1: 15.2213
[850]	valid_0's l1: 14.4392	valid_1's l1: 15.1874
[900]	valid_0's l1: 14.3865	valid_1's l1: 15.1613
[950]	valid_0's l1: 14.3407	valid_1's

In [25]:
pred_mae = 0
for i,model in enumerate(models_mae): 
    pred_mae += model.predict(consumerTest[fea_cols]) * 0.2
consumerTest['pred_mae'] = pred_mae

In [26]:
pred_mse = 0
for i,model in enumerate(models_mse): 
    pred_mse += model.predict(consumerTest[fea_cols]) * 0.2
consumerTest['pred_mse'] = pred_mse

In [27]:
consumerTest = consumerTest.sort_values('pred_mae')
consumerTest['ranks'] = list(range(consumerTest.shape[0]))
consumerTest['score'] = consumerTest['pred_mae'].values
consumerTest.loc[consumerTest.ranks<10000,'score']  = consumerTest.loc[consumerTest.ranks< 10000,'pred_mse'].values *0.4 + consumerTest.loc[consumerTest.ranks< 10000,'pred_mae'].values * 0.6
consumerTest.loc[consumerTest.ranks>40000,'score']  = consumerTest.loc[consumerTest.ranks> 40000,'pred_mse'].values *0.4 + consumerTest.loc[consumerTest.ranks> 40000,'pred_mae'].values * 0.6   

In [28]:
submit_mae_mse = pd.DataFrame()
submit_mae_mse['id']    = consumerTest['用户编码'].values
submit_mae_mse['score'] = consumerTest['score'].values 
submit_mae_mse['score'] = submit_mae_mse['score'].astype(int)
submit_mae_mse[['id','score']].to_csv('baseline_mae_mse.csv',index = None)
submit_mae_mse['score'].describe()

count    50000.000000
mean       618.591820
std         37.873375
min        471.000000
25%        598.000000
50%        628.000000
75%        646.000000
max        694.000000
Name: score, dtype: float64

In [29]:
print("LOVE")

LOVE


In [30]:
# from lightgbm import LGBMRegressor 


In [31]:
# from xgboost import XGBRegressor
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import AdaBoostRegressor
# from sklearn import model_selection

# x = consumerTrain.drop(["信用分","用户编码"],axis=1)
# y = consumerTrain.信用分
# x_train,x_test, y_train,y_test = train_test_split(x,y,test_size=0.2, random_state=10)


In [32]:
# clf_est =LGBMRegressor(random_state=1314)

# clf_param_grid = {'n_estimators': [1500],
#                   'learning_rate': [0.08],
#                  'max_depth':[5],
#                  'bagging_fraction':[0.4],
#                   'min_child_weight':[4],
#                   'num_leaves':[40],
#                   'alpha':[0.1] #
#                  }

# clf_grid = model_selection.GridSearchCV(clf_est, clf_param_grid, n_jobs=4, cv=2, verbose=1)
# print("fitting")
# clf_grid.fit(x_train, y_train)

# print('BestParams: ' + str(clf_grid.best_params_))
# print('Training:')
# clf = LGBMRegressor(n_estimators = clf_grid.best_estimator_.n_estimators,
#                    learning_rate = clf_grid.best_estimator_.learning_rate,
#                     max_depth = clf_grid.best_estimator_.max_depth,
#                     bagging_fraction = clf_grid.best_estimator_.bagging_fraction,
#                     min_child_weight = clf_grid.best_estimator_.min_child_weight,
# #                     max_bin = clf_grid.best_estimator_.max_bin,
#                     num_leaves = clf_grid.best_estimator_.num_leaves,
#                     alpha = clf_grid.best_estimator_.alpha,
#                    random_state=1314)

# clf.fit(x_train, y_train)
# print('Training Finished')

In [33]:

# predictions = clf.predict(consumerTest.astype(int))
# Submission = pd.DataFrame({'id': Id, 
#                            'score': predictions})
# Submission["score"] = Submission["score"].astype(int)
# Submission.to_csv('SubmissionLightGBM.csv',index=False,sep=',')
# print("I LOVE YOUU")

# print("评分：")
# from sklearn.model_selection import cross_val_score

# scores = cross_val_score(clf,x_test,y_test)
# print("LGB: ",scores.mean())
# from sklearn import metrics

# y_pred_xgb = clf.predict(x_test)
# # 均方根误差
# MSE = metrics.mean_squared_error(y_test,y_pred_xgb)
# print(MSE)

In [34]:
# print(1/(1+MSE))